In [35]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [36]:
data_folder = f"../data"

sample_filename = "muestreos_parcelas.parquet"

df_samples = pd.read_parquet(f"{data_folder}/{sample_filename}")

df_samples["fecha"] = pd.to_datetime(df_samples["fecha"])


df_samples["codparcela"] = df_samples["codparcela"].str.replace("/", "")

df_samples.info()

<class 'pandas.core.frame.DataFrame'>
Index: 581793 entries, 0 to 581792
Data columns (total 61 columns):
 #   Column                                              Non-Null Count   Dtype         
---  ------                                              --------------   -----         
 0   generated_muestreos                                 581793 non-null  datetime64[us]
 1   codparcela                                          581793 non-null  object        
 2   provincia                                           581793 non-null  string        
 3   municipio                                           581793 non-null  string        
 4   fecha                                               581793 non-null  datetime64[us]
 5   campaña                                             581793 non-null  uint16        
 6   poligono                                            581731 non-null  UInt16        
 7   parcela                                             581763 non-null  UInt16        
 8  

In [37]:
parcela_filename = "parcelas_download.txt"
df_parcelas = pd.read_csv(f"{data_folder}/{parcela_filename}", sep="\t", index_col=0, usecols=["codparcela", "lat", "lon"])
df_parcelas.head()

,lat,lon
codparcela,,
015-00002-00-00,37.802069,-3.984598
043-00008-00-00,36.944813,-5.301941
017-00204-00-00,38.052148,-3.312451
010-00016-00-00,38.070431,-3.509408
007-00018-01-01,38.043972,-4.463010


In [42]:
#df_samples  = pd.merge(df_samples, df_parcelas, on='codparcela', how='left')
df_samples.head()

,generated_muestreos,codparcela,provincia,municipio,fecha,campaña,poligono,parcela,recinto,subrecinto,...,108_u_h_c_a_la_que_pertenece,316_fecha_de_plantación_variedad_secundaria,315_patrón_variedad_secundaria,317_%_superficie_ocupada_variedad_secundaria,306_altura_de_copa_(m),310_patrón_variedad_principal,411_representa_a_la_u_h_c_(si/no),109_sistema_para_el_cumplimiento_gestión_integrada,lat,lon
0,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-04-16,2006,23,109,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.146121,-2.769372
1,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-04-27,2006,23,109,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.146121,-2.769372
2,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-04,2006,23,109,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.146121,-2.769372
3,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-11,2006,23,109,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.146121,-2.769372
4,2020-04-23 14:30:39,023-00109-00-00,almeria,abla,2006-05-17,2006,23,109,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.146121,-2.769372


In [57]:
parcelas_with_data = [file.split("_BIOPAR")[0] for file in os.listdir("../data/raw/NVDI_FAPAR_data") if "NDVI" in file]

df_samples = df_samples[df_samples["codparcela"].isin(parcelas_with_data)]

In [61]:
df_samples.to_parquet("../data/subset_muestreos_parcelas.parquet")

In [108]:
parcelas_with_data = [file.split("_BIOPAR")[0] for file in os.listdir("../data/raw/NVDI_FAPAR_data") if "NDVI" in file]

for id_parcela in parcelas_with_data:

    datos_parcela = [file for file in os.listdir("../data/NVDI_FAPAR_data") if id_parcela in file]

    ndvi_filename = next((file for file in datos_parcela if "NDVI" in file), None)
    fapar_filename = next((file for file in datos_parcela if "FAPAR" in file), None)

    df_ndvi = pd.read_csv(f"../data/NVDI_FAPAR_data/{ndvi_filename}", sep="\t", index_col=0)
    df_fapar = pd.read_csv(f"../data/NVDI_FAPAR_data/{fapar_filename}", sep="\t", index_col=0)

    df_merge = pd.merge(df_ndvi, df_fapar, on='date', how='outer', suffixes=('_ndvi', '_fapar')).sort_values(by="date").reset_index(drop=True)

    df_merge.to_csv(f"../data/clean/NVDI_FAPAR_data/{id_parcela}.csv", sep="\t")